In [4]:
#coding=utf-8
%matplotlib ipympl
import sys
sys.path.insert(0, '../')

from AircraftIden.StateSpaceIden import StateSpaceIdenSIMO, StateSpaceParamModel

from AircraftIden import FreqIdenSIMO, TransferFunctionFit
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import sympy as sp
pkl_name = "../../../FLYLOG/foam-tail-sitter/cruising/log_34_2018-4-10-freq-res.pkl"
finput = open(pkl_name, 'rb')
freq_responses = pickle.load(finput)
def plot_freqres_for_status(freqres , key = "-"):
    
    plt.figure("ele_q")
    freqres.plt_bode_plot(0, label=key)
    plt.figure("ele_body_vx")
    freqres.plt_bode_plot(1, label=key)
    plt.figure("ele_body_vz")
    freqres.plt_bode_plot(2, label=key)
    plt.figure("ele_body_ax")
    freqres.plt_bode_plot(3, label=key)
    plt.figure("ele_body_az")
    freqres.plt_bode_plot(4, label=key)
    plt.show()

In [5]:
plt.rc('figure', figsize=(10.0, 5.0))
plt.close('all')
freqres_10ms = freq_responses["10m/s"]
plot_freqres_for_status(freqres_10ms)

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

..\AircraftIden\FreqIden.py:158: RuntimeWarning: invalid value encountered in log10
  plt.semilogx(freq, 20 * np.log10(gxx), label=label+'gxx')


FigureCanvasNbAgg()

In [6]:
# X = [u,w,q,th]
# Y = [u,w,q]

M = sp.Matrix([[1, 0, 0, 0],
               [0, 1 , 0, 0],
               [0, 0, 1, 0],
               [0, 0, 0, 1]])

g = 9.78
U0 = 9.0
W0 = 0
th0 = 30 / 180 * math.pi

print("Trim theta {} U0 {} W0 {}".format(th0, U0, W0))
Xu, Xw, Xq = sp.symbols('Xu Xw Xq')
Zu, Zw, Zq = sp.symbols('Zu Zw Zq')
Mu, Mw, Mq = sp.symbols('Mu Mw Mq')

F = sp.Matrix([[Xu, Xw, Xq - W0, -g * sp.cos(th0)],
               [Zu, Zw, Zq + U0, -g * sp.sin(th0)],
               [Mu, Mw, Mq, 0],
               [0, 0, 1, 0]])

Xele, Zele, Mele = sp.symbols('Xele,Zele,Mele')
G = sp.Matrix([[Xele],
               [Zele],
               [Mele],
               [0]])

# direct using u w q for y
H0 = sp.Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],    
    [0, 0, 1, 0]]
)

H1 = sp.Matrix([
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
])

syms = [
        Xu, Xw, Xq,
        Zu, Zw, Zq,
        Mu, Mw, Mq,
        Xele, Zele, Mele
]
LatdynSSPM = StateSpaceParamModel(M, F, G, H0, H1, syms)

Trim theta 0.5235987755982988 U0 9.0 W0 0


In [7]:
plt.rc('figure', figsize=(10.0, 5.0))
freqres = freqres_10ms.get_freqres(indexs = [2,1,0])
ssm_iden = StateSpaceIdenSIMO(freqres, accept_J=50,
                              enable_debug_plot=True,
                              y_names=["V", "w", "q"],reg=0 )
J, ssm = ssm_iden.estimate(LatdynSSPM, syms, constant_defines={})
ssm.check_stable()
ssm_iden.draw_freq_res()
ssm_iden.print_res()

Will estimate num 12 [A_0_0, A_0_1, A_0_2, A_1_0, A_1_1, A_1_2, A_2_0, A_2_1, A_2_2, B_0_0, B_1_0, B_2_0]
Found new better 101.04504685215794
Found new better 31.36857310941146
Terminate pool
Using J 31.36857310941146 x [ -10.23167552    8.66631063    3.38558607  -10.87496096    8.39218206
    4.80405075 -528.57334121  360.56294059  -20.67792966   -7.91066998
   -3.70863886  -66.36651836]
{Xu: -10.2316755160090, Xw: 8.66631062905871, Xq: 3.38558607095214, Zu: -10.8749609568715, Zw: 8.39218205852207, Zq: -4.19594924759957, Mu: -528.573341213109, Mw: 360.562940592937, Mq: -20.6779296553808, Xele: -7.91066997770445, Zele: -3.70863885982562, Mele: -66.3665183570009}
J : 31.36857310941146 syms {Xu: -10.2316755160090, Xw: 8.66631062905871, Xq: 3.38558607095214, Zu: -10.8749609568715, Zw: 8.39218205852207, Zq: -4.19594924759957, Mu: -528.573341213109, Mw: 360.562940592937, Mq: -20.6779296553808, Xele: -7.91066997770445, Zele: -3.70863885982562, Mele: -66.3665183570009}


FigureCanvasNbAgg()

Eigs [-1.60525828e+01+0.        j -6.46374057e+00+0.        j
 -5.49874040e-04+0.00212537j -5.49874040e-04-0.00212537j]
Stable


FigureCanvasNbAgg()

{Xu: -10.2316755160090, Xw: 8.66631062905871, Xq: 3.38558607095214, Zu: -10.8749609568715, Zw: 8.39218205852207, Zq: -4.19594924759957, Mu: -528.573341213109, Mw: 360.562940592937, Mq: -20.6779296553808, Xele: -7.91066997770445, Zele: -3.70863885982562, Mele: -66.3665183570009}
{Xu: -10.2316755160090, Xw: 8.66631062905871, Xq: 3.38558607095214, Zu: -10.8749609568715, Zw: 8.39218205852207, Zq: -4.19594924759957, Mu: -528.573341213109, Mw: 360.562940592937, Mq: -20.6779296553808, Xele: -7.91066997770445, Zele: -3.70863885982562, Mele: -66.3665183570009}
A
[[ -10.23167552    8.66631063    3.38558607   -8.46972845]
 [ -10.87496096    8.39218206    4.80405075   -4.89      ]
 [-528.57334121  360.56294059  -20.67792966    0.        ]
 [   0.            0.            1.            0.        ]]
B
[[ -7.91066998]
 [ -3.70863886]
 [-66.36651836]
 [  0.        ]]
